In [ ]:
!pip install deepspeech

     |████████████████████████████████| 9.2 MB 5.5 MB/s 


In [9]:
!pip install datasets

     |████████████████████████████████| 298 kB 4.4 MB/s 
     |████████████████████████████████| 132 kB 51.8 MB/s 
     |████████████████████████████████| 61 kB 557 kB/s 
     |████████████████████████████████| 243 kB 36.6 MB/s 
     |████████████████████████████████| 1.1 MB 32.6 MB/s 
     |████████████████████████████████| 160 kB 43.5 MB/s 
     |████████████████████████████████| 192 kB 28.9 MB/s 
     |████████████████████████████████| 271 kB 47.5 MB/s 


In [ ]:
!pip install editdistance

#Pre-Processing

In [10]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

Downloading:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import deepspeech

In [ ]:
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm

In [ ]:
model = deepspeech.Model('/deepspeech-0.9.3-models.pbmm')

#DeepSpeech

In [14]:
import wave
import numpy as np
import editdistance

def DeepSpeechInference(n,model):
  seqs_hat = []
  seqs_true = []
  for i in range(n):

    path = timit['test']['audio'][i]['path']+'.wav'
    w = wave.open(path, 'r')

    frames = w.getnframes()
    buffer = w.readframes(frames)
    data16 = np.frombuffer(buffer, dtype=np.int16)
    seqs_hat.append(model.stt(data16))
    seqs_true.append(timit['test']['text'][i])
  return seqs_hat, seqs_true


def calculate_wer(seqs_hat, seqs_true):
  word_eds, word_ref_lens = [], []
  for i, seq_hat_text in enumerate(seqs_hat):
      seq_true_text = seqs_true[i]
      hyp_words = seq_hat_text.split()
      ref_words = seq_true_text.split()

      word_eds.append(editdistance.eval(hyp_words, ref_words))

      word_ref_lens.append(len(ref_words))
  return float(sum(word_eds)) / sum(word_ref_lens) 


In [ ]:
seqs_hat, seqs_true = DeepSpeechInference(1680,model)

In [ ]:
seqs_hat = [ i.lower() for i in seqs_hat]
seqs_true = [ i.lower() for i in seqs_true]

In [16]:
calculate_wer(seqs_hat, seqs_true)

0.38235294117647056

#Load_Results

In [7]:
%cd /content

/content


In [11]:
import pickle
file_name = "deepspeech_pred"
open_file = open(file_name, "rb")
seqs_hat = pickle.load(open_file)
open_file.close()


In [15]:
seqs_hat = [ i.lower() for i in seqs_hat]
calculate_wer(seqs_hat,[i.lower() for i in timit['test']['text']])

0.38235294117647056

#WavetoVec2.0

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.0 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained('facebook/wav2vec2-base-960h')
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')


Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
model.to('cuda')
model.device

device(type='cuda', index=0)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

In [ ]:
import torch

def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

  0%|          | 0/1680 [00:00<?, ?ex/s]

In [ ]:
!pip install jiwer

     |████████████████████████████████| 50 kB 3.1 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149863 sha256=507a3f0c5fe6e42ad955dad9801c066378a43b76e92e9ff561019fa5c39b1fae
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [17]:
timit = load_dataset("timit_asr")

Reusing dataset timit_asr (/root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/5bebea6cd9df0fc2c8c871250de23293a94c1dc49324182b330b6759ae6718f8)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
a = [i.lower() for i in results["pred_str"]]
b = [i.lower() for i in timit['test']['text']]

In [ ]:
calculate_wer(a, b)

0.23963355834136935